In [1]:
import sys
import time
from pyclick.click_models.Evaluation import LogLikelihood, Perplexity
from pyclick.click_models.UBM import UBM
from pyclick.click_models.DBN import DBN
from pyclick.click_models.SDBN import SDBN
from pyclick.click_models.DCM import DCM
from pyclick.click_models.CCM import CCM
from pyclick.click_models.CTR import DCTR, RCTR, GCTR
from pyclick.click_models.CM import CM
from pyclick.click_models.PBM import PBM
from pyclick.utils.Utils import Utils
from pyclick.click_models.task_centric.SearchTask import SearchTask
from pyclick.click_models.task_centric.TCM import TCM
from pyclick.utils.YandexRelPredChallengeParser import YandexRelPredChallengeParser
from pyclick.utils.YandexPersonalizedChallengeParser import YandexPersonalizedChallengeParser

click_model = PBM()
search_sessions_path = "data/YandexPersonalizedChallenge"
search_sessions_num = 1000

#search_sessions = YandexRelPredChallengeParser().parse(search_sessions_path, search_sessions_num)
search_sessions = YandexPersonalizedChallengeParser().parse(search_sessions_path, search_sessions_num) #for YandexPersonalizedChallengeParser

train_test_split = int(len(search_sessions) * 0.75)
train_sessions = search_sessions[:train_test_split]
train_queries = Utils.get_unique_queries(train_sessions)
train_tasks = SearchTask.get_search_tasks(train_sessions) #for tcm model

test_sessions = Utils.filter_sessions(search_sessions[train_test_split:], train_queries)
test_queries = Utils.get_unique_queries(test_sessions)
test_tasks = SearchTask.get_search_tasks(test_sessions) #for tcm model

print("-------------------------------")
print("Training on %d search sessions (%d unique queries)." % (len(train_sessions), len(train_queries)))
print("-------------------------------")

start = time.time()
click_model.train(train_sessions)
#click_model.train(train_tasks)  #for tcm model
end = time.time()
print("\tTrained %s click model in %i secs:\n%r" % (click_model.__class__.__name__, end - start, click_model))

print("-------------------------------")
print("Testing on %d search sessions (%d unique queries)." % (len(test_sessions), len(test_queries)))
print("-------------------------------")

loglikelihood = LogLikelihood()
perplexity = Perplexity()

start = time.time()
ll_value = loglikelihood.evaluate(click_model, test_sessions)
end = time.time()
print("\tlog-likelihood: %f; time: %i secs" % (ll_value, end - start))

start = time.time()
perp_value = perplexity.evaluate(click_model, test_sessions)[0]
end = time.time()
print("\tperplexity: %f; time: %i secs\n" % (perp_value, end - start))

print(test_sessions)
print(test_queries)
print(test_tasks)

-------------------------------
Training on 750 search sessions (677 unique queries).
-------------------------------
	Trained PBM click model in 4 secs:
PBMParamNames.attr
18896602: {'28045079': 0.4963, '66222069': 0.6667, '66326309': 0.4987, '52334111': 0.4970, '12236869': 0.6667, '43571721': 0.4963, '58910663': 0.6667, '54099223': 0.6667, '39621370': 0.6667, '50020626': 0.4966}
7600975: {'47872329': 0.4956, '60485854': 0.4890, '64426805': 0.4800, '45114654': 0.4518, '59181983': 0.4970, '13090448': 0.4963, '5787912': 0.4987, '8255310': 0.4966, '29361827': 0.4963, '36085243': 0.4949}
15293650: {'70164926': 0.4966, '63238535': 0.4949, '67160294': 0.4970, '25723512': 0.4963, '4340589': 0.4987, '48069615': 0.4890, '45025328': 0.4963, '8584957': 0.4956, '22102711': 0.4518, '70153895': 0.4800}
18899864: {'51413154': 0.4963, '14669307': 0.4890, '47427536': 0.4956, '39622388': 0.4970, '58931164': 0.4800, '51038152': 0.4987, '15804246': 0.4966, '47608980': 0.4963, '47609932': 0.4518, '1223494